# SK: Agents using Planner

## Outline:

* Using built in SK Skills: HttpSkill, TimeSkill and MathSkill
* Use other skills examples in SK (WriterSkill) from a directory. This is how you can write your own skill and make it available to the agent

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [26]:
# Standard Semantic Kernel initialization
import semantic_kernel as sk
import os
import logging
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('__name__')
kernel=sk.Kernel(log=logger)

api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key)
)

## Built-in SK tools

In [27]:
# SK comes with a few built in skills/plugins. Lets use those and have SK generate a plan for which skills to use for a prompt
kernel.import_skill(sk.core_skills.MathSkill(), "MathSkill")
kernel.import_skill(sk.core_skills.TimeSkill(), "TimeSkill")
kernel.import_skill(sk.core_skills.HttpSkill(), "HttpSkill")

{'deleteAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f1b69d8ea30>,
 'getAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f1b69d8e760>,
 'postAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f1b69d8ecd0>,
 'putAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f1b69d8ee20>}

In [28]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [29]:
ask = "What is the sum of 2319 and 86988?"

In [30]:
# For this above "ask" the planner shoudl choose the mathskill
original_plan = await planner.create_plan_async(ask, kernel)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=983 request_id=6c52cd873cd3f227451385c085f3359f response_code=200


In [31]:
# Lets see what the plan says
print(original_plan.generated_plan)

{
    "input": "2319 and 86988",
    "subtasks": [
        {"function": "MathSkill.Add", "args": {"input": 2319, "Amount": 86988}}
    ]
}


In [32]:
# Plan looks good if it returned mathSkill. Now execute it
results = await planner.execute_plan_async(original_plan, kernel)
print(results)

89307


In [33]:
ask="whats the date today?"
plan2 = await planner.create_plan_async(ask, kernel)
print(plan2.generated_plan)
result2= await planner.execute_plan_async(plan2, kernel)
print(result2)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=662 request_id=58197b79e1e2e311490c249f9616c3cc response_code=200


{
    "input": "today's date",
    "subtasks": [
        {"function": "TimeSkill.now"}
    ]
}
Thursday, July 06, 2023 04:09 PM


In [34]:
# Plans generated may not always be accurate. 
ask = "What book did Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU) write?"

# IMPORTANT: Copy the skills/WriterSkill directory from https://github.com/microsoft/semantic-kerne into current directory
skills_directory = "skills"
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")

plan3 = await planner.create_plan_async(ask, kernel)
print(plan3.generated_plan)
 

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1753 request_id=3da7ac2b06a161ba0a2daf066e78de41 response_code=200


{
    "input": "Tom M. Mitchell",
    "subtasks": [
        {"function": "HttpSkill.getAsync", "args": {"input": "https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Tom_M._Mitchell&prop=extracts&exintro&explaintext"}},
        {"function": "WriterSkill.Rewrite", "args": {"style": "compact"}}
    ]
}


In [35]:
result3= await planner.execute_plan_async(plan3, kernel)
print(result3)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1765 request_id=228f404ff276dd630ce14530c808cf0b response_code=200


Tom M. Mitchell, an American computer scientist, is the Founders University Professor at Carnegie Mellon University (CMU). He founded and chaired the Machine Learning Department at CMU and is renowned for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience. Mitchell authored the textbook Machine Learning and has been a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science, and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.
